In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
금융 데이터 수집 모듈
- 외부 금융 데이터 수집 (Naver Finance, KRX, FnGuide 등)
- 데이터 정제 및 저장
"""

import os
import requests
import pandas as pd
import json
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any, Tuple, Optional

class DataCollector:
    """금융 데이터 수집 클래스"""
    def __init__(self):
        self.script_dir = os.path.dirname(os.path.abspath(__file__))
        self.data_dir = os.path.join(os.path.dirname(self.script_dir), "data")
        self.raw_data_dir = os.path.join(self.data_dir, "raw")
        self.processed_data_dir = os.path.join(self.data_dir, "processed")
        
        # 디렉토리 초기화
        os.makedirs(self.raw_data_dir, exist_ok=True)
        os.makedirs(self.processed_data_dir, exist_ok=True)
        
        # 헤더 설정
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def collect_stock_tickers(self):
        """주식 종목 코드 수집"""
        try:
            print("주식 종목 코드 수집 중...")
            
            # 네이버 금융에서 종목 코드 수집
            # KOSPI
            url_kospi = "https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt"
            df_kospi = pd.read_html(url_kospi, header=0)[0]
            df_kospi['시장구분'] = 'KOSPI'
            
            # KOSDAQ
            url_kosdaq = "https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=kosdaqMkt"
            df_kosdaq = pd.read_html(url_kosdaq, header=0)[0]
            df_kosdaq['시장구분'] = 'KOSDAQ'
            
            # 데이터 결합
            df = pd.concat([df_kospi, df_kosdaq])
            
            # 컬럼명 변경 및 종목코드 포맷 조정
            df = df.rename(columns={
                '회사명': '종목명',
                '종목코드': '종목코드',
                '업종': '섹터',
                '주요제품': '주요제품'
            })
            
            # 종목코드 6자리로 포맷팅
            df['종목코드'] = df['종목코드'].astype(str).str.zfill(6)
            
            # 파일 저장
            today = datetime.now().strftime('%Y%m%d')
            output_file = os.path.join(self.raw_data_dir, f"kor_ticker_{today}.csv")
            df.to_csv(output_file, index=False, encoding='utf-8')
            
            print(f"종목 코드 수집 완료: {len(df)}개 종목, 저장 경로: {output_file}")
            return df
        except Exception as e:
            print(f"종목 코드 수집 실패: {e}")
            return None
    
    def collect_stock_prices(self, tickers=None, days=30):
        """주식 가격 데이터 수집"""
        try:
            print("주식 가격 데이터 수집 중...")
            
            # 종목 코드 로드
            if tickers is None:
                ticker_files = sorted([f for f in os.listdir(self.raw_data_dir) if f.startswith('kor_ticker_')], reverse=True)
                if not ticker_files:
                    print("종목 코드 파일이 없습니다. 종목 코드 수집을 먼저 실행하세요.")
                    return None
                
                ticker_file = os.path.join(self.raw_data_dir, ticker_files[0])
                ticker_df = pd.read_csv(ticker_file)
                tickers = ticker_df['종목코드'].tolist()
            
            # 날짜 범위 설정
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)
            
            # 결과 데이터프레임 초기화
            all_prices = []
            
            # 진행 상황 표시
            total_tickers = len(tickers)
            for i, ticker in enumerate(tickers[:100]):  # 테스트를 위해 100개만 수집
                try:
                    print(f"[{i+1}/{total_tickers}] {ticker} 가격 데이터 수집 중...")
                    
                    # 네이버 금융에서 가격 데이터 수집
                    url = f"https://api.finance.naver.com/siseJson.naver?symbol={ticker}&requestType=1&startTime={start_date.strftime('%Y%m%d')}&endTime={end_date.strftime('%Y%m%d')}&timeframe=day"
                    response = requests.get(url, headers=self.headers)
                    
                    # 응답 데이터 처리
                    data = response.text.replace("'", '"').strip()
                    data = json.loads(data)
                    
                    # 데이터프레임 생성
                    columns = data[0]
                    price_data = data[1:]
                    df = pd.DataFrame(price_data, columns=columns)
                    
                    # 컬럼명 변경
                    df = df.rename(columns={
                        '날짜': '날짜',
                        '시가': '시가',
                        '고가': '고가',
                        '저가': '저가',
                        '종가': '종가',
                        '거래량': '거래량'
                    })
                    
                    # 종목코드 추가
                    df['종목코드'] = ticker
                    
                    # 날짜 형식 변환
                    df['날짜'] = pd.to_datetime(df['날짜'].astype(str), format='%Y%m%d')
                    
                    # 결과 추가
                    all_prices.append(df)
                    
                    # API 과부하 방지
                    time.sleep(0.1)
                except Exception as e:
                    print(f"  - {ticker} 데이터 수집 실패: {e}")
                    continue
            
            # 모든 데이터 결합
            if all_prices:
                result_df = pd.concat(all_prices, ignore_index=True)
                
                # 파일 저장
                today = datetime.now().strftime('%Y%m%d')
                output_file = os.path.join(self.raw_data_dir, f"kor_price_{today}.csv")
                result_df.to_csv(output_file, index=False, encoding='utf-8')
                
                print(f"가격 데이터 수집 완료: {len(result_df)}개 레코드, 저장 경로: {output_file}")
                return result_df
            else:
                print("수집된 가격 데이터가 없습니다.")
                return None
        except Exception as e:
            print(f"가격 데이터 수집 실패: {e}")
            return None
    
    def collect_financial_statements(self, tickers=None):
        """재무제표 데이터 수집"""
        try:
            print("재무제표 데이터 수집 중...")
            
            # 종목 코드 로드
            if tickers is None:
                ticker_files = sorted([f for f in os.listdir(self.raw_data_dir) if f.startswith('kor_ticker_')], reverse=True)
                if not ticker_files:
                    print("종목 코드 파일이 없습니다. 종목 코드 수집을 먼저 실행하세요.")
                    return None
                
                ticker_file = os.path.join(self.raw_data_dir, ticker_files[0])
                ticker_df = pd.read_csv(ticker_file)
                tickers = ticker_df['종목코드'].tolist()
            
            # 결과 데이터프레임 초기화
            all_fs = []
            
            # 진행 상황 표시
            total_tickers = len(tickers)
            for i, ticker in enumerate(tickers[:50]):  # 테스트를 위해 50개만 수집
                try:
                    print(f"[{i+1}/{total_tickers}] {ticker} 재무제표 데이터 수집 중...")
                    
                    # FnGuide에서 재무제표 데이터 수집 (실제로는 API 키가 필요할 수 있음)
                    # 여기서는 간단한 샘플 데이터 생성
                    years = [2020, 2021, 2022, 2023]
                    quarters = [1, 2, 3, 4]
                    
                    for year in years:
                        for quarter in quarters:
                            # 샘플 데이터 생성
                            fs_data = {
                                '종목코드': ticker,
                                '연도': year,
                                '분기': quarter,
                                '매출액': int(1000000 * (1 + 0.1 * (year - 2020) + 0.02 * quarter)),
                                '영업이익': int(100000 * (1 + 0.15 * (year - 2020) + 0.03 * quarter)),
                                '당기순이익': int(80000 * (1 + 0.12 * (year - 2020) + 0.025 * quarter)),
                                '자산총계': int(5000000 * (1 + 0.05 * (year - 2020))),
                                '부채총계': int(2500000 * (1 + 0.03 * (year - 2020))),
                                '자본총계': int(2500000 * (1 + 0.07 * (year - 2020))),
                                'EPS': int(5000 * (1 + 0.1 * (year - 2020) + 0.02 * quarter)),
                                'BPS': int(30000 * (1 + 0.07 * (year - 2020))),
                                'ROE': 15.0 + 0.5 * (year - 2020) + 0.1 * quarter,
                                'ROA': 7.0 + 0.3 * (year - 2020) + 0.05 * quarter,
                                '부채비율': 50.0 - 0.5 * (year - 2020)
                            }
                            all_fs.append(fs_data)
                    
                    # API 과부하 방지
                    time.sleep(0.1)
                except Exception as e:
                    print(f"  - {ticker} 데이터 수집 실패: {e}")
                    continue
            
            # 모든 데이터 결합
            if all_fs:
                result_df = pd.DataFrame(all_fs)
                
                # 파일 저장
                today = datetime.now().strftime('%Y%m%d')
                output_file = os.path.join(self.raw_data_dir, f"kor_fs_{today}.csv")
                result_df.to_csv(output_file, index=False, encoding='utf-8')
                
                print(f"재무제표 데이터 수집 완료: {len(result_df)}개 레코드, 저장 경로: {output_file}")
                return result_df
            else:
                print("수집된 재무제표 데이터가 없습니다.")
                return None
        except Exception as e:
            print(f"재무제표 데이터 수집 실패: {e}")
            return None
    
    def collect_valuation_metrics(self, tickers=None):
        """주식 가치평가 지표 수집"""
        try:
            print("주식 가치평가 지표 수집 중...")
            
            # 종목 코드 로드
            if tickers is None:
                ticker_files = sorted([f for f in os.listdir(self.raw_data_dir) if f.startswith('kor_ticker_')], reverse=True)
                if not ticker_files:
                    print("종목 코드 파일이 없습니다. 종목 코드 수집을 먼저 실행하세요.")
                    return None
                
                ticker_file = os.path.join(self.raw_data_dir, ticker_files[0])
                ticker_df = pd.read_csv(ticker_file)
                tickers = ticker_df['종목코드'].tolist()
            
            # 결과 데이터프레임 초기화
            all_values = []
            
            # 진행 상황 표시
            total_tickers = len(tickers)
            for i, ticker in enumerate(tickers[:100]):  # 테스트를 위해 100개만 수집
                try:
                    print(f"[{i+1}/{total_tickers}] {ticker} 가치평가 지표 수집 중...")
                    
                    # 네이버 금융에서 가치평가 지표 수집 (실제로는 API 키가 필요할 수 있음)
                    # 여기서는 간단한 샘플 데이터 생성
                    value_data = {
                        '종목코드': ticker,
                        'PER': round(15.0 + 10.0 * (ticker[0] == '0'), 2),  # 샘플 PER
                        'PBR': round(1.5 + 0.5 * (ticker[0] == '0'), 2),    # 샘플 PBR
                        'PCR': round(10.0 + 5.0 * (ticker[0] == '0'), 2),   # 샘플 PCR
                        'PSR': round(1.2 + 0.3 * (ticker[0] == '0'), 2),    # 샘플 PSR
                        'EV/EBITDA': round(8.0 + 2.0 * (ticker[0] == '0'), 2),  # 샘플 EV/EBITDA
                        '배당수익률': round(2.0 + 1.0 * (ticker[0] == '0'), 2)    # 샘플 배당수익률
                    }
                    all_values.append(value_data)
                    
                    # API 과부하 방지
                    time.sleep(0.1)
                except Exception as e:
                    print(f"  - {ticker} 데이터 수집 실패: {e}")
                    continue
            
            # 모든 데이터 결합
            if all_values:
                result_df = pd.DataFrame(all_values)
                
                # 파일 저장
                today = datetime.now().strftime('%Y%m%d')
                output_file = os.path.join(self.raw_data_dir, f"kor_value_{today}.csv")
                result_df.to_csv(output_file, index=False, encoding='utf-8')
                
                print(f"가치평가 지표 수집 완료: {len(result_df)}개 레코드, 저장 경로: {output_file}")
                return result_df
            else:
                print("수집된 가치평가 지표가 없습니다.")
                return None
        except Exception as e:
            print(f"가치평가 지표 수집 실패: {e}")
            return None
    
    def collect_sector_data(self):
        """섹터별 종목 데이터 수집"""
        try:
            print("섹터별 종목 데이터 수집 중...")
            
            # 종목 코드 로드
            ticker_files = sorted([f for f in os.listdir(self.raw_data_dir) if f.startswith('kor_ticker_')], reverse=True)
            if not ticker_files:
                print("종목 코드 파일이 없습니다. 종목 코드 수집을 먼저 실행하세요.")
                return None
            
            ticker_file = os.path.join(self.raw_data_dir, ticker_files[0])
            ticker_df = pd.read_csv(ticker_file)
            
            # 섹터 정보가 있는지 확인
            if '섹터' in ticker_df.columns:
                # 섹터별로 그룹화
                sector_data = []
                for sector, group in ticker_df.groupby('섹터'):
                    for _, row in group.iterrows():
                        sector_data.append({
                            '종목코드': row['종목코드'],
                            '종목명': row['종목명'],
                            'sector': sector
                        })
                
                # 데이터프레임 생성
                result_df = pd.DataFrame(sector_data)
                
                # 파일 저장
                today = datetime.now().strftime('%Y%m%d')
                output_file = os.path.join(self.raw_data_dir, f"kor_sector_{today}.csv")
                result_df.to_csv(output_file, index=False, encoding='utf-8')
                
                print(f"섹터별 종목 데이터 수집 완료: {len(result_df)}개 레코드, 저장 경로: {output_file}")
                return result_df
            else:
                print("종목 코드 파일에 섹터 정보가 없습니다.")
                return None
        except Exception as e:
            print(f"섹터별 종목 데이터 수집 실패: {e}")
            return None
    
    def run_all_collectors(self):
        """모든 데이터 수집 함수 실행"""
        print("모든 데이터 수집 시작...")
        
        # 종목 코드 수집
        ticker_df = self.collect_stock_tickers()
        
        if ticker_df is not None:
            # 주식 가격 데이터 수집
            self.collect_stock_prices(tickers=ticker_df['종목코드'].tolist()[:100])
            
            # 재무제표 데이터 수집
            self.collect_financial_statements(tickers=ticker_df['종목코드'].tolist()[:50])
            
            # 가치평가 지표 수집
            self.collect_valuation_metrics(tickers=ticker_df['종목코드'].tolist()[:100])
            
            # 섹터별 종목 데이터 수집
            self.collect_sector_data()
        
        print("모든 데이터 수집 완료")

    def collect_us_stock_tickers(self):
        """
        S&P 500 티커 리스트만 자동 수집 및 저장
        - yfinance의 tickers_sp500()만 활용
        - 결과: data/raw/us_ticker_YYYYMMDD.csv 저장
        - 주요 컬럼: Ticker, Name, Market, Sector(가능시)
        """
        try:
            import yfinance as yf
            print("S&P 500 티커 리스트 수집 중...")
            sp500 = yf.tickers_sp500()
            tickers = list(set(sp500))
            records = []
            for i, ticker in enumerate(tickers):
                try:
                    info = yf.Ticker(ticker).info
                    name = info.get('shortName', '')
                    market = info.get('exchange', '')
                    sector = info.get('sector', '')
                    records.append({'Ticker': ticker, 'Name': name, 'Market': market, 'Sector': sector})
                    if i % 50 == 0:
                        print(f"{i+1}/{len(tickers)}개 처리 중...")
                    time.sleep(0.01)
                except Exception as e:
                    print(f"  - {ticker} 정보 조회 실패: {e}")
                    continue
            import pandas as pd
            df = pd.DataFrame(records)
            today = datetime.now().strftime('%Y%m%d')
            output_file = os.path.join(self.raw_data_dir, f"us_ticker_{today}.csv")
            df.to_csv(output_file, index=False, encoding='utf-8')
            print(f"S&P 500 티커 리스트 수집 완료: {len(df)}개, 저장 경로: {output_file}")
            return df
        except Exception as e:
            print(f"S&P 500 티커 리스트 수집 실패: {e}")
            return None

    def collect_us_stock_prices(self, tickers=None, days=1):
        """
        S&P 500 일별 가격 데이터 수집 (yfinance)
        - tickers: S&P 500 티커 리스트(없으면 자동으로 yfinance.tickers_sp500() 사용)
        - days: 최근 n일치(기본 1일, 당일)
        - 결과: data/raw/us_price_YYYYMMDD.csv 저장
        - 주요 컬럼: Date, Open, High, Low, Close, Volume, Ticker
        """
        try:
            import yfinance as yf
            print("S&P 500 가격 데이터 수집 중...")
            if tickers is None:
                tickers = yf.tickers_sp500()
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days-1)
            all_prices = []
            for i, ticker in enumerate(tickers):
                try:
                    print(f"[{i+1}/{len(tickers)}] {ticker} 가격 데이터 수집 중...")
                    df = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=(end_date+timedelta(days=1)).strftime('%Y-%m-%d'), interval='1d')
                    if not df.empty:
                        df = df.reset_index()
                        df['Ticker'] = ticker
                        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']]
                        all_prices.append(df)
                    time.sleep(0.1)
                except Exception as e:
                    print(f"  - {ticker} 데이터 수집 실패: {e}")
                    continue
            if all_prices:
                import pandas as pd
                result_df = pd.concat(all_prices, ignore_index=True)
                today = datetime.now().strftime('%Y%m%d')
                output_file = os.path.join(self.raw_data_dir, f"us_price_{today}.csv")
                result_df.to_csv(output_file, index=False, encoding='utf-8')
                print(f"S&P 500 가격 데이터 수집 완료: {len(result_df)}개 레코드, 저장 경로: {output_file}")
                return result_df
            else:
                print("수집된 S&P 500 가격 데이터가 없습니다.")
                return None
        except Exception as e:
            print(f"S&P 500 가격 데이터 수집 실패: {e}")
            return None

# 모듈 테스트용 코드
if __name__ == "__main__":
    collector = DataCollector()
    collector.run_all_collectors() 